In [9]:
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('85c168c81886441d89c30d0bd8613cc0')

In [95]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm
scl_colormap = np.array(
    [
        [252,  40, 228, 255],  # 0  - NODATA - MAGENTA
        [255,   0,   4, 255],  # 1  - Saturated or Defective - RED
        [0  ,   0,   0, 255],  # 2  - Dark Areas - BLACK
        [97 ,  97,  97, 255],  # 3  - Cloud Shadow - DARK GREY
        [3  , 139,  80, 255],  # 4  - Vegetation - GREEN
        [192, 132,  12, 255],  # 5  - Bare Ground - BROWN
        [21 , 103, 141, 255],  # 6  - Water - BLUE
        [117,   0,  27, 255],  # 7  - Unclassified - MAROON
        [208, 208, 208, 255],  # 8  - Cloud - LIGHT GREY
        [244, 244, 244, 255],  # 9  - Definitely Cloud - WHITE
        [195, 231, 240, 255],  # 10 - Thin Cloud - LIGHT BLUE
        [222, 157, 204, 255],  # 11 - Snow or Ice - PINK
    ],
    dtype="uint8",
)
resolution = 20 # meters per pixel
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 

In [33]:
# Create a mask for no data, saturated data, clouds, cloud shadows, and water
def create_cloud_mask(xx):
    cloud_mask = \
        (xx.SCL != 0) & \
        (xx.SCL != 1) & \
        (xx.SCL != 3) & \
        (xx.SCL != 6) & \
        (xx.SCL != 8) & \
        (xx.SCL != 9) & \
        (xx.SCL != 10) 
    return cloud_mask

In [34]:
crop_presence_data = pd.read_csv("Crop_Location_Data.csv")
box_size_deg = 0.000898 # Surrounding box in degrees, 5x5 pixels, 20m per pixel

In [85]:
def further_remove_cloud(items):
    i = 0
    removal_list = []
    while i < len(items):
        if items[i].properties['eo:cloud_cover'] > 60:
            removal_list.append(i)
        i += 1
    k = 0
    for k in range(len(removal_list)):
        items.pop(removal_list[k]-k)
    return items

In [121]:
def get_ndvi_data(latlong, time):
    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    min_lat = float(latlong[0]) - box_size_deg/2
    min_long = float(latlong[1]) - box_size_deg/2
    max_lat = float(latlong[0]) + box_size_deg/2
    max_long = float(latlong[1]) + box_size_deg/2
    bbox_of_interest = (min_long, min_lat, max_long, max_lat)
    time_of_interest = time
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox_of_interest, datetime=time_of_interest)
    items = list(search.get_all_items())
    items = further_remove_cloud(items)
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL"],
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bbox_of_interest
    )
    cleaned_data = xx.where(create_cloud_mask(xx)).astype("uint16")
    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    ndvi_mean_clean = (mean_clean.nir-mean_clean.red)/(mean_clean.nir+mean_clean.red)
    ndvi_mean_clean=ndvi_mean_clean.to_numpy()
    ndvi_mean_clean = ndvi_mean_clean[~np.isnan(ndvi_mean_clean)]
    try:
        return max(ndvi_mean_clean), min(ndvi_mean_clean)
    except ValueError:
        return 0,0

In [ ]:
time = "2021-12-01/2022-04-30"
max_ndvis = []
min_ndvis = []
#max_ndvi, min_ndvi = get_ndvi_data(crop_presence_data['Latitude and Longitude'].iloc[400], time)
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    max_ndvi, min_ndvi = get_ndvi_data(coordinates, time)
    max_ndvis.append(max_ndvi)
    min_ndvis.append(min_ndvi)
    

crop_presence_data['max_ndvi'] = max_ndvis
crop_presence_data['min_ndvi'] = min_ndvis
crop_presence_data.to_csv('max_min_crop_data.csv', index=False)

 88%|████████▊ | 528/600 [15:26<02:06,  1.75s/it]